!git clone https://github.com/ultralytics/yolov5  # clone repo


In [ ]:
%cd yolov5
!git reset --hard 064365d8683fd002e9ad789c1e91fa3d021b44f0

!pip install -qr requirements.txt  # install dependencies (ignore errors)


In [ ]:
!pip install utils

In [ ]:
import torch

from IPython.display import Image, clear_output  # to display images

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

In [ ]:
!pip install roboflow



from roboflow import Roboflow
rf = Roboflow(api_key="qsriDCqyPmufazKUFp0g")
project = rf.workspace("project-ssayl").project("potholes-detection-d4rma")
dataset = project.version(1).download("yolov5")


import yaml
with open("Potholes-Detection-1/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [ ]:
import os
HOME = os.getcwd()

In [ ]:
{HOME}

%cat /{HOME}/Potholes-Detection-1/data.yaml

from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

%%writetemplate {HOME}/models/custom_yolov5s.yaml
nc: 1  

depth_multiple: 0.33  
width_multiple: 0.50  

anchors:
  - [10,13, 16,30, 33,23]
  - [30,61, 62,45, 59,119] 
  - [116,90, 156,198, 373,326] 
backbone:
  [[-1, 1, Focus, [64, 3]],  
   [-1, 1, Conv, [128, 3, 2]],  
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  
  ]


head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],
   [-1, 3, BottleneckCSP, [512, False]], 

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],
   [-1, 3, BottleneckCSP, [256, False]],

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  
   [-1, 3, BottleneckCSP, [512, False]],  

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  
   [-1, 3, BottleneckCSP, [1024, False]], 

   [[17, 20, 23], 1, Detect, [nc, anchors]],  
  ]

In [ ]:
!pip install tensorboard
import tensorboard

%%time
%cd {HOME}/
!python train.py --img 416 --batch 16 --epochs 100 --data Potholes-Detection-1/data.yaml --cfg ./models/custom_yolov5s.yaml --weights '' --name yolov5s_res  --cache

In [ ]:
!python detect.py --weights {HOME}/runs/train/yolov5s_res/weights/best.pt --img 416 --conf 0.4 --source Potholes-Detection-1/test/images

In [ ]:
ls

In [4]:
from models.yolo import Model  
model_yaml = Model("models/custom_yolov5s.yaml")





                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Focus                     [3, 32, 3]                    
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     19904  models.common.BottleneckCSP             [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  3    161152  models.common.BottleneckCSP             [128, 128, 3]                 
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  6                -1  3    641792  models.common.BottleneckCSP             [256, 256, 3]                 
  7                -1  1   1180672  models.common.Conv                      [256, 512, 3, 2]              
  8                -1  1    656896  

In [2]:
cd yolov5

/home/ec2-user/SageMaker/yolov5


In [3]:
import torch
model = torch.load("runs/train/yolov5s_res/weights/best.pt")

In [ ]:
model

In [17]:
from PIL import Image
image_path = "3.jpg"
image = Image.open(image_path)

In [19]:
import torchvision.transforms as transforms

preprocess = transforms.Compose([
    transforms.Resize((416, 416)),
    transforms.ToTensor(),
])

In [20]:
input_image = preprocess(image).unsqueeze(0)

In [21]:
with torch.no_grad():
    results = model_yaml(input_image)


In [22]:
results

[tensor([[[[[-6.00144e-02, -1.60872e-02, -8.71957e-02,  1.51425e-01, -6.69995e+00,  1.09977e+01],
            [-2.13738e-01, -1.62406e-01, -4.48569e-02,  1.21956e-02, -6.66976e+00,  1.09818e+01],
            [-1.46911e-01, -1.48971e-01,  3.71879e-02,  3.46353e-02, -6.66309e+00,  1.09454e+01],
            ...,
            [-2.51737e-01, -3.80478e-02,  1.31652e-01,  1.69859e-01, -6.66064e+00,  1.06859e+01],
            [ 7.32756e-02, -8.81650e-02, -5.40111e-02,  1.16586e-01, -6.49956e+00,  1.08788e+01],
            [-6.26929e-02,  1.12337e-01, -4.34923e-02,  3.05795e-01, -6.53086e+00,  1.09566e+01]],
 
           [[-7.01220e-02, -9.36858e-02, -6.76813e-02,  7.26860e-02, -6.74917e+00,  1.10492e+01],
            [-1.48798e-01, -1.22382e-01, -6.60297e-02, -1.88902e-04, -6.67029e+00,  1.10740e+01],
            [-1.00326e-01, -1.62901e-01,  1.11941e-02,  6.83247e-03, -6.61543e+00,  1.11231e+01],
            ...,
            [-1.44086e-01,  2.00617e-02,  3.62046e-02,  8.67069e-02, -6.78683e+00

In [32]:
#bbox (x_max, y_max, x_min, y_min), obj score and class represent the columns in the tensor

In [25]:
len(results)

3

In [34]:
ls

3.jpg                  Document.jpg           requirements.txt
benchmarks.py          export.py              runs/
CITATION.cff           {HOME}/                segment/
classify/              hubconf.py             setup.cfg
CONTRIBUTING.md        LICENSE                train.py
data/                  models/                tutorial.ipynb
detection_result0.txt  Potholes-Detection-1/  utils/
detection_result1.txt  __pycache__/           val.py
detection_result2.txt  README.md              yolov5s.pt
detect.py              README.zh-CN.md


In [36]:
!python export.py --weights runs/train/yolov5s_res/weights/best.pt --include onnx


export: data=data/coco128.yaml, weights=['runs/train/yolov5s_res/weights/best.pt'], imgsz=[640, 640], batch_size=1, device=cpu, half=False, inplace=False, keras=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=17, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['onnx']
YOLOv5 🚀 v7.0-72-g064365d Python-3.10.12 torch-2.0.1 CPU

Fusing layers... 
custom_YOLOv5s summary: 182 layers, 7246518 parameters, 0 gradients

PyTorch: starting from runs/train/yolov5s_res/weights/best.pt with output shape (1, 25200, 6) (14.1 MB)

ONNX: starting export with onnx 1.13.1...
================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

ONNX: export success ✅ 4.5s, saved as runs/train/yolov5s_res/weights/best.onnx (28.1 MB)

Export complete (7.1s)
Results saved

In [39]:
!python export.py --weights runs/train/yolov5s_res/weights/best.pt --include tflite

export: data=data/coco128.yaml, weights=['runs/train/yolov5s_res/weights/best.pt'], imgsz=[640, 640], batch_size=1, device=cpu, half=False, inplace=False, keras=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=17, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['tflite']
YOLOv5 🚀 v7.0-72-g064365d Python-3.10.12 torch-2.0.1 CPU

Fusing layers... 
custom_YOLOv5s summary: 182 layers, 7246518 parameters, 0 gradients

PyTorch: starting from runs/train/yolov5s_res/weights/best.pt with output shape (1, 25200, 6) (14.1 MB)
requirements: YOLOv5 requirement "tensorflow-cpu" not found, attempting AutoUpdate...
  Obtaining dependency information for tensorflow-cpu from https://files.pythonhosted.org/packages/c3/be/dd4f4a51f25af1973c8a87598672748c64bf98c69182dce846d955577f00/tensorflow_cpu-2.14.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency info